In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
import pandas as pd
import seaborn as sns; print(sns.__version__)
from datetime import datetime
import pingouin as pg
data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))

print(data_dir_input, scripts_dir)

subjects = [f for f in os.listdir(data_dir_input) if f.startswith('sub') ]
subjects = list(sorted(subjects))
print(subjects)

### prep

In [ ]:
from config2 import path_data
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from behav_proc import *

import numpy as np
#from collections import OrderedDict as odict
from config2 import envcode2env
from pingouin import ttest

from base2 import radius_cursor
radius_cursor
ps_2nice = dict( zip(['pre','pert','washout','rnd'], 
        ['No perturbation','Perturbation','Washout','Random']) )

target_angs = (np.array([157.5, 112.5, 67.5, 22.5]) + 90) * (np.pi / 180)
print(target_angs / np.pi * 180)

import behav_proc
from behav_proc import addBehavCols2, truncateNIHDfFromES, calcESthr

In [ ]:
%%javascript
// needed to access notebook path later
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

In [ ]:
import os

def getAddInfo():
    import datetime
    # Get the current date and time in the format YYYY-MM-DD HH:MM:SS
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # Get the name of the current jupyter notebook
    #notebook_name = os.path.basename(os.environ.get("NOTEBOOK_PATH", ""))
    notebook_name = NOTEBOOK_FULL_PATH
    #print(notebook_name)
    add_ttle = f"{now}\n{notebook_name}"
    return add_ttle

def addTitleInfo(axs):
    add_ttle = getAddInfo()
    # Add a linebreak and the date, time, and notebook name to the ylabel of each axis
    if axs is not None:
        if isinstance(axs, (list, np.ndarray)): # If axs is a list or an array of axes
            for ax in axs:
                title = ax.get_title()
                new_title = f"{title}\n{add_ttle}"
                ax.set_title(new_title)
        else: # If axs is a single axis
            title = axs.get_title()
            new_title = f"{title}\n{add_ttle}"
            axs.set_title(new_title)
    else:
        title = plt.gcf()._suptitle.get_text()
        new_title = f"{title}\n{add_ttle}"
        plt.suptitle(new_title)

In [ ]:
print("NOTEBOOK_FULL_PATH:\n", NOTEBOOK_FULL_PATH)

In [ ]:
fnf = pjoin(path_data,'df_all_multi_tsz_.pkl.zip')
#fnf = '/home/demitau/ownCloud/Current/merr_data/df_all_multi_tsz.pkl.zip'
print(fnf)
print( str(datetime.fromtimestamp(os.stat(fnf).st_mtime)))
df_all_multi_tsz = pd.read_pickle(fnf)
#dfc_multi_tsz = pd.   read_pickle(pjoin(path_data,'df_all_multi_tsz_trunc_q=0.05.pkl.zip'))
# df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
#                             ' and retention_factor_s == "1.000"').copy()

df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
                            ' and retention_factor_s == "0.924"').copy()
df_orig = df.copy()

In [ ]:
df,dfall,ES_thr,envv,pert = addBehavCols2(df);
df_wthr = df

In [ ]:
%matplotlib inline
sns.set(font_scale=1.3)

# Variability (compare with Tan)

In [ ]:
#dfc = dfall_aug.query('thr == "mestd*1.0"')
#dfc = dfall.query('thr == "mestd*1.0"')
# here we need all trials, w/o holes
dfc = df_wthr.copy()

## corr ES and variance (and other statistics)

### fixed histlen across subjects

In [ ]:
from behav_proc import addWindowStatCols
dfcs,dfcs_fixhistlen,dfcs_fixhistlen_untrunc,histlens  = addWindowStatCols(dfc, ES_thr, 
                                    varn0s = ['error','error_pscadj','error_pscadj_abs'])
dfcs_fixhistlen, me_pct_excl = behav_proc.truncLargeStats(dfcs_fixhistlen_untrunc, histlens, 5.)

In [ ]:
np.std([0])

In [ ]:
len(me_pct_excl)

In [ ]:
me_pct_excl.query('mean_excl >= 1')

me_pct_excl['mean_excl'].describe([0.95])

#### stats

In [ ]:
from behav_proc import compare0
import pingouin as pg

In [ ]:
sm.__version__

In [ ]:
dfcs_fixhistlen['environment'] = dfcs_fixhistlen['environment'].astype(int)
df_ = dfcs_fixhistlen[['environment','subject','trials','error_pscadj_abs_Tan29']]
assert not df_.duplicated(['subject','trials']).any()
#df_.to_csv('/home/demitau/dfcs.csv')

In [ ]:
histlens

In [ ]:
dfcs_fixhistlen.columns

In [ ]:
# test one
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings

#result.pvalues, result.params are pd.Series objects containing actual values (possibly nan)
# z is something like std (in fact it is coef / std)

d={}
varn = 'error_pscadj_abs_Tan20'
df_ = dfcs_fixhistlen.dropna(subset = varn)
model = smf.mixedlm(f"err_sens ~ C(environment) + {varn} + C(environment) * {varn}", df_, 
                    groups=df_["subject"], re_formula="~C(environment)")
d['inter_randomeff'] = model

model = smf.mixedlm(f"err_sens ~ C(environment) + {varn} + C(environment) * {varn}", df_, 
                    groups=df_["subject"], re_formula="1")
d['inter'] = model

model = smf.mixedlm(f"err_sens ~ C(environment) + {varn} + C(environment) * {varn}", df_, 
                    groups=df_["subject"], re_formula="~C(environment)")
d['nointer_randomeff'] = model

model = smf.mixedlm(f"err_sens ~ C(environment) + {varn}", df_, 
                    groups=df_["subject"])
d['nointer'] = model
del model

# disp=True is not helpful for catching warnings
res = {}
for mn,model in d.items():
    wmess = []
    with warnings.catch_warnings(record = True) as w:
        result = model.fit(full_output=True)
        for warning in w:
            wmess += [warning.message]
            print(warning.message)
    result.wmess = wmess

    result.converged2 = result.converged and ( not (result.params.isna().any() \
                                                        | result.pvalues.isna().any()) )
    res[mn] = result

    
for mn,result in res.items():
#print('result.converged2 = ',result.converged2)
    summ = result.summary()
    summ.tables[0].loc[5,2] = 'Converged2:'
    summ.tables[0].loc[5,3] = 'Yes' if result.converged2 else 'No'


    print(mn)
    display(summ)

#from statsmodels.tools.sm_exceptions import ConvergenceWarning
#with warnings.catch_warnings():
    #warnings.filterwarnings('ignore',category=ConvergenceWarning)


In [ ]:
import gc
gc.collect()

In [ ]:
#dfcs_fixhistlen.pre_break_duration

In [ ]:
#dfcs_fixhistlen.loc[dfcs_fixhistlen.was_pre_break,
#                    ['trial_index','pre_break_duration','err_sens']]

In [ ]:
all_suffixes = 'mav,std,invstd,mavsq,mav_d_std,mav_d_var,Tan,invmavsq,invmav,std_d_mav,invTan'.split(',')

varn0 = 'error_pscadj'
n = 3
for suffix in all_suffixes:
    s = f'{varn0}_{suffix}{n}'
    if s not in dfcs_fixhistlen.columns:
        print(s)
    assert s in dfcs_fixhistlen.columns

print( all_suffixes )

In [ ]:
prl = []

In [ ]:
# TODO: make prev error abs

In [ ]:
# run long calc
from joblib import Parallel, delayed
from itertools import product
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning    
from numpy.linalg import LinAlgError
import traceback
import datetime
gc.collect()
n_jobs_inside = 1

# Define the function to be executed in parallel
def run_model(args, ret_res = False):
    dfcs_fixhistlen, cocoln, std_mavsz_, varn0, varn_suffix = args
    varn = f'{varn0}_{varn_suffix}{std_mavsz_}'
    df_ = dfcs_fixhistlen.dropna(subset=[varn, 'err_sens'])
    df_ = df_[~np.isinf(df_[varn])]

    excfmt = None
    nstarts = 1
    result = None
    if cocoln == 'None':
        s,s2 = f"err_sens ~ {varn}","1"
        model = smf.mixedlm(s, df_, 
                    groups=df_["subject"])
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore',category=ConvergenceWarning)
            result = model.fit()
        results = {(s,s2): result}
    else:
        flas = []
        s,s2 = f"err_sens ~ C({cocoln}) + {varn} + C({cocoln}) * {varn} + {varn} * prev_error_pscadj_abs",\
            f"~C({cocoln})"; flas += [(s,s2)]
        s,s2 = f"err_sens ~ C({cocoln}) + {varn} + C({cocoln}) * {varn} + {varn} * prev_error_pscadj_abs",\
            f"1"; flas += [(s,s2)]
        s,s2 = f"err_sens ~ C({cocoln}) + {varn} + C({cocoln}) * {varn}", f"~C({cocoln})"; flas += [(s,s2)]
        s,s2 = f"err_sens ~ C({cocoln}) + {varn} + C({cocoln}) * {varn}","1";  flas += [(s,s2)]
        
        s,s2 = f"err_sens ~ C({cocoln}) + {varn}",f"~C({cocoln})"; flas += [(s,s2)]
        s,s2 = f"err_sens ~ C({cocoln}) + {varn}","1"; flas += [(s,s2)]        
        
     
        results = {}
        for s,s2 in flas:
            try:                
                model = smf.mixedlm(s, df_.copy(), 
                            groups=df_["subject"], re_formula=s2)
                with warnings.catch_warnings(record=True) as w:
                    ###warnings.filterwarnings('ignore',category=ConvergenceWarning)     
                    # n_jobs argument does not really work :(
                    result = model.fit(n_jobs =n_jobs_inside)
                    wmess = []
                    for warning in w:
                        wmess += [warning.message]
                    result.converged2 = result.converged and \
                        ( not (result.params.isna().any() | result.pvalues.isna().any()) )
                    result.wmess = wmess

            except LinAlgError as le:
                excfmt = traceback.format_exc()
                result = None
            results[(s,s2)] = result
        
    s2summary = {}
    for stpl,result in results.items():        
        if (result is not None) and result.converged:
            #result.remove_data()
            summary = result.summary()
            summary.tables[0].loc[5,2] = 'Converged2:'
            summary.tables[0].loc[5,3] = 'Yes' if result.converged2 else 'No'
            summary.wmess = result.wmess
            summary.params = result.params
            summary.pvalues = result.pvalues
        else:
            summary = None
        s2summary[stpl] = summary
    print(args[1:])
    r = {'cocoln': cocoln, 'histlen': std_mavsz_,
            'varn': varn, 'varn0':varn0, 'varn_suffix':varn_suffix,             
             'excfmt':excfmt,
            's2summary': s2summary, 'retention_factor':df_.iloc[0]['retention_factor_s']}
            #'res': result}
            #'nstarts':nstarts,
    if ret_res:
        r['s2res'] = results
    return r

N = 25
# Create args array using product
cocols = ['None', 'env', 'ps2_']
#cocols = [ 'env'] #, 'ps2_']
#std_mavsz_range = range(2, 30)
std_mavsz_range = list(range(3,N)) + list( range(N, 40, 3) )
#std_mavsz_range = range(2, 20)
#std_mavsz_range = range(2, 12)
#std_mavsz_range = range(2, 3)
varn0s = ['error_pscadj', 'error_pscadj_abs']
#varn_suffixes = ['std', 'invstd', 'mavsq', 'mav_d_std', 'mav_d_var', 'Tan']
varn_suffixes = all_suffixes

# shorter
cocols = [ 'env', 'ps2_']
varn0s = ['error'] #, 'error_pscadj', 'error_pscadj_abs']
#varn_suffixes = ['std', 'invstd', 'Tan']
#N_ = 10
#std_mavsz_range = list(range(3,N_)) + list( range(N_, 32, 3) )

# #std_mavsz_range = range(2, 15)
# std_mavsz_range = range(2, 30)
# varn0s = ['error_pscadj_abs']
# #varn0s = ['error_pscadj', 'error_pscadj_abs']
# #cocols = [ 'env', 'ps2_']
# #cocols = [ 'ps2_']
# cocols = [ 'env']

args = list(product(cocols, std_mavsz_range, varn0s, varn_suffixes))
print(len(args))
# Number of processes
#n_jobs = 10  # Use all available CPUs even with one job
#n_jobs = 5
n_jobs = 1

ind = 0
if n_jobs > 1:
    # Execute in parallel
    backend = 'multiprocessing' # 'loky'
    #backend = 'loky' 
    prl = Parallel(n_jobs=n_jobs, backend = backend)\
        (delayed(run_model)( (dfcs_fixhistlen,*arg) ) for arg in args)
else:
    for arg in args:
        prl += [run_model((dfcs_fixhistlen,*arg))]
#     for arg in args[69:69+1]:
#         prl += [run_model((dfcs_fixhistlen,*arg),ret_res=True)]
        
        if len(prl) >= 100:            
            s_ = str(datetime.datetime.now())[:-7].replace(' ','_')
            np.savez( pjoin(path_data, f'prl_{ind}_{s_}'), prl )
            ind += 1
            del prl
            gc.collect()
            prl = []
    # to save the last
    s_ = str(datetime.datetime.now())[:-7].replace(' ','_')
    np.savez( pjoin(path_data, f'prl_{ind+1}_{s_}'), prl )


gc.collect()

In [ ]:
print(len(args))

In [ ]:
ind

In [ ]:
dif = (dfcs_fixhistlen['error_pscadj_std2'] - dfcs_fixhistlen['error_pscadj_abs_std2'])#
display(dif.describe())
dif.hist()

In [ ]:
#dfcs_fixhistlen['_dif'] = dif
dfcs_fixhistlen[['error_pscadj','error_pscadj_abs', 'error_pscadj_std2','error_pscadj_abs_std2','_dif']]

In [ ]:
import config2

In [ ]:
res = prl[0]['s2res'][('err_sens ~ C(env) + error_pscadj_abs_mav_d_std7 + C(env) * error_pscadj_abs_mav_d_std7',
  '~C(env)')]
print( res.params )
print( res.pvalues )

In [ ]:
prl

# Tests

In [ ]:
import numpy as np

# Define parameters for the distributions
poisson_lambda = 5  # Mean for Poisson distribution
normal_mean = 5  # Mean for normal distribution
normal_std = 2  # Standard deviation for normal distribution
N = 5000

# Generate Poisson random variable
poisson_var = np.random.gamma(poisson_lambda,size=N)

# Generate normal random variable
normal_var = np.random.normal(normal_mean, normal_std,size=N)

# Print the generated variables
print("Poisson variable:", poisson_var)
print("Normal variable:", normal_var)

import pandas as pd
a = np.array(list(zip(normal_var,poisson_var)))
dft = pd.DataFrame( a, columns=['normal','poisson'])
    

corr = pg.corr(dft['normal'], dft['poisson'], method='spearman')
display(corr)

sns.relplot(data=dft, x='normal',y='poisson')

In [ ]:
# the Shepherd pi uses a
# bootstrapping of the Mahalanobis distance to identify outliers, while the
# skipped correlation is based on the minimum covariance determinant
# (which requires scikit-learn). Note that these two methods are
# significantly slower than the previous ones.

# The `biweight midcorrelation
# <https://en.wikipedia.org/wiki/Biweight_midcorrelation>`_ and
# percentage bend correlation [1]_ are both robust methods that
# protects against *univariate* outliers by down-weighting observations that
# deviate too much from the median

corr = pg.corr(dft['normal'], dft['poisson'], method='shepherd')
display(corr.reset_index())